# Tabelas Municípios Brasileiros

Esse notebook tem como objetivo criar o arquivo municipios.csv com as colunas:

- codigo_regiao
- sigla_regiao
- nome_regiao
- codigo_uf
- sigla_uf
- nome_uf
- codigo_regiao_intermediaria
- nome_regiao_intermediaria
- codigo_regiao_imediata
- nome_regiao_imediata
- codigo_municipio
- nome_municipio
- capital

In [1]:
import pandas as pd

In [2]:
def export(df, file_path_wo_extension):
    !mkdir -p $(dirname {file_path_wo_extension})

    df.to_csv(
        file_path_wo_extension + '.csv',
        index=None,
        encoding='utf-8',
    )

    df.to_json(
        file_path_wo_extension + '.json',
        orient='records',
        force_ascii=False,
    )

In [15]:
ufs_raw = pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/estados')
ufs_raw.head()

,id,nome,regiao,sigla
0,11,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}",RO
1,12,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}",AC
2,13,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}",AM
3,14,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}",RR
4,15,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}",PA


In [30]:
regioes = (
    pd
    .DataFrame(
        ufs_raw.regiao.values.tolist(),
        columns=['id', 'sigla', 'nome'],
    )
    .drop_duplicates()
    .sort_values('id')
    .rename(columns={'id': 'codigo_regiao', 'nome': 'nome_regiao', 'sigla': 'sigla_regiao'})
    .reset_index(drop=True)
)

regioes

,codigo_regiao,sigla_regiao,nome_regiao
0,1,N,Norte
1,2,NE,Nordeste
2,3,SE,Sudeste
3,4,S,Sul
4,5,CO,Centro-Oeste


In [48]:
ufs = pd.DataFrame({
    'codigo_uf': ufs_raw.id,
    'sigla_uf': ufs_raw.sigla,
    'nome_uf': ufs_raw.nome,
    'codigo_regiao': ufs_raw.regiao.map(lambda d: d['id']),
}).sort_values('codigo_uf')

ufs.head()

,codigo_uf,sigla_uf,nome_uf,codigo_regiao
0,11,RO,Rondônia,1
1,12,AC,Acre,1
2,13,AM,Amazonas,1
3,14,RR,Roraima,1
4,15,PA,Pará,1


In [36]:
municipios_raw = pd.read_excel(
    'ftp://geoftp.ibge.gov.br/organizacao_do_territorio/divisao_regional/divisao_regional_do_brasil/divisao_regional_do_brasil_em_regioes_geograficas_2017/tabelas/regioes_geograficas_composicao_por_municipios_2017_20180911.xlsx'
).rename(columns={
    'nome_mun': 'nome_municipio',
    'CD_GEOCODI': 'codigo_municipio',
    'cod_rgi': 'codigo_regiao_imediata',
    'nome_rgi': 'nome_regiao_imediata',
    'cod_rgint': 'codigo_regiao_intermediaria',
    'nome_rgint': 'nome_regiao_intermediaria',
}).assign(codigo_uf=lambda df: df.codigo_regiao_intermediaria // 100)

municipios_raw.head()

,nome_municipio,codigo_municipio,codigo_regiao_imediata,nome_regiao_imediata,codigo_regiao_intermediaria,nome_regiao_intermediaria,codigo_uf
0,Itapuã do Oeste,1101104,110001,Porto Velho,1101,Porto Velho,11
1,Candeias do Jamari,1100809,110001,Porto Velho,1101,Porto Velho,11
2,Nova Mamoré,1100338,110001,Porto Velho,1101,Porto Velho,11
3,Porto Velho,1100205,110001,Porto Velho,1101,Porto Velho,11
4,Guajará-Mirim,1100106,110001,Porto Velho,1101,Porto Velho,11


In [38]:
regioes_intermediarias = municipios_raw[[
    'codigo_regiao_intermediaria',
    'nome_regiao_intermediaria',
    'codigo_uf'
]].drop_duplicates()

regioes_intermediarias.head()

,codigo_regiao_intermediaria,nome_regiao_intermediaria,codigo_uf
0,1101,Porto Velho,11
18,1102,Ji-Paraná,11
52,1201,Rio Branco,12
66,1202,Cruzeiro do Sul,12
74,1301,Manaus,13


In [39]:
regioes_imediatas = municipios_raw[[
    'codigo_regiao_imediata',
    'nome_regiao_imediata',
    'codigo_regiao_intermediaria'
]].drop_duplicates()

regioes_imediatas.head()

,codigo_regiao_imediata,nome_regiao_imediata,codigo_regiao_intermediaria
0,110001,Porto Velho,1101
5,110002,Ariquemes,1101
13,110003,Jaru,1101
18,110004,Ji-Paraná,1102
31,110005,Cacoal,1102


In [41]:
municipios = municipios_raw[[
    'codigo_municipio',
    'nome_municipio',
    'codigo_regiao_imediata',
]]

municipios.head()

,codigo_municipio,nome_municipio,codigo_regiao_imediata
0,1101104,Itapuã do Oeste,110001
1,1100809,Candeias do Jamari,110001
2,1100338,Nova Mamoré,110001
3,1100205,Porto Velho,110001
4,1100106,Guajará-Mirim,110001


In [47]:
capitais = (
    pd
    .read_csv(
        'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/8730f8f9f90a47546f08b216484f5c336a1f531a/municipios_brasileiros.csv'
    )
    .loc[lambda df: df.capital == 1, 'codigo_ibge']
    .rename('codigo_municipio')
    .to_frame()
)


capitais.head()

,codigo_municipio
294,2800308
580,1501402
592,3106200
642,1400100
755,5300108
